# Web Scraping by CVR code
For culture-den cultural classifier 

In [2]:
import os
import csv
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

import selenium


In [3]:
from parsel import Selector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [13]:
#Read in CSV to get some sample CVRs
oracle = pd.read_csv("oracledf.csv")
oracle.columns = ['ID', 'Affiliation', "TAGS", "CVR"]
oracle.head(n = 10)

,ID,Affiliation,TAGS,CVR
0,50935,Dansk Kennelklub (bestyrelse),"Recreation and sports, Recreation, Pets, Animals",FALSE
1,55276,SPAREKASSEN BORNHOLMS FOND,"Foundation, Charity, Banks",15106891
2,45889,Bager- og Konditormestre i Danmark (BKD) (best...,"Employers association, Food",FALSE
3,47447,Statens Kunstfond (Billedkunstudvalget),"Culture, Foundation, Art",FALSE
4,28654,Dansk Aktionærforening (Generalforsamlingsudvalg),"Business association, Investment, Owners, Finance",FALSE
5,55126,"VOGNMAND JOHS. RASMUSSEN, SVEBØLLE OG RUTH RAS...","Foundation, Owners, Corporation",82447016
6,36512,Sanistål,"Corporation, ENGH",42997811
7,56094,Dansk Petanque Forbund (forretningsudvalg),"Recreation and sports, Sports",FALSE
8,31607,Udvalg om bygningsbevaring,"Commission, City planning, History, Housing, A...",FALSE
9,42966,GPV International,"Corporation, MASK",66945715


Base URL: https://datacvr.virk.dk/
Example with CVR: https://datacvr.virk.dk/enhed/virksomhed/21378607?fritekst=21378607&sideIndex=0&size=10

In [30]:
def purposeScrapeSel(cvr):
    """Takes CVR number for organization, returns purpose statement (in Danish) from https://datacvr.virk.dk/
    Args:
        cvr (numeric): CVR number from den$CVR
    """

    options = Options()
    #options.headless = True

    driver = webdriver.Chrome('/Users/miriamfeldman/Desktop/chromedriver', options = options)

    #Construct Full URL from CVR Number
    url1 = 'https://datacvr.virk.dk/enhed/virksomhed/'
    url2 = '?fritekst='
    url3 = '&sideIndex=0&size=10'

    urlfull = url1+str(cvr)+url2+str(cvr)+url3 
    #print(urlfull)

    #Access web page with Chrome driver
    driver.get(urlfull)

    #Wait until loaded (by when the header appears)
    try:
        element = WebDriverWait(driver=driver, timeout=10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'h1'))
            )
        
        #Get HTML source code
        html_source = driver.page_source

        #Load into BeautifulSoup to parse the HTML
        soup = BeautifulSoup(html_source, 'html.parser')
    
    except:
        purpose_text = ''

    #Find the column with label "Formål"
    #Should eventually add a try-except to catch ones that don't have a purpose statement
    try:
        cols = soup.find(class_ = "col-12 col-lg-3", text = re.compile('Formål'))

        parent_tag = cols.parent #Go up one level
        purpose_statement = parent_tag.findNext(class_ = 'col') #Find next 'col' where purpose text stored
        purpose_text = purpose_statement.contents[0] #Text of purpose statement
    
    except:
        purpose_text = ''

    driver.close()

    return purpose_text


### Test Scraping

In [4]:
test1 = purposeScrapeSel(15106891)
print(test1)

Fondens formål er at virke til gavn for almennyttige og velgørende formål på Bornholm, alt efter bestyrelsens frie skøn.


DeepL Translation: The purpose of the Foundation is to work for the benefit of non-profit and charitable purposes on Bornholm, at the discretion of the Board.

In [5]:
test2 = purposeScrapeSel(21378607)
print(test2)

Selskabets formål er handel, produktion, distribution, udvikling og markedsføring indenfor audio og dermed beslægtet virksomhed, dels i eget regi, dels gennem koncernforbundne datterselskaber.


DeepL Translation: The object of the company is the trade, production, distribution, development and marketing of audio and related activities, partly under its own management and partly through group subsidiaries.

In [29]:
cvrs = pd.read_csv("affilsandcvrs.csv")
cvrs.columns = ['Affiliation', "CVR"]
cvrs.head(n = 100)


,Affiliation,CVR
0,Reception på Kongeskibet Dannebrog (2012),FALSE
1,Middelfart Sparekasse,24744817
2,Foreningen Østifterne - Repræsentantskab (Medl...,FALSE
3,Regentparrets haveselskab (for frivillige arbe...,FALSE
4,Rude Strand Seniorhøjskole (Bestyrelse),FALSE
...,...,...
95,TEAM DANMARK (Aktivkomitéen),FALSE
96,Danske Regioner (bestyrelse),FALSE
97,Danske Regioner (direktionen),FALSE
98,"AKF, Anvendt KommunalForskning (bestyrekse)",FALSE


## Scrape in batches

In [45]:
cvr_text_1000 = []

for i in range(0, 1000):
    if i%100 == 0:
        print(i)
    if cvrs.iloc[i,1] == "FALSE":
        cvr_text_1000.append('')
    else:
        cvr_text_1000.append(purposeScrapeSel(cvrs.iloc[i,1]))


0
100
200
300
400
500
600
700
800
900


In [48]:
df = pd.DataFrame(cvr_text_1000) 
    
# saving the dataframe 
df.to_csv('cvr_text_1000.csv')

In [49]:
cvr_text_2000 = []

for i in range(1000, 2000):
    if i%100 == 0:
        print(i)
    if cvrs.iloc[i,1] == "FALSE":
        cvr_text_2000.append('')
    else:
        cvr_text_2000.append(purposeScrapeSel(cvrs.iloc[i,1]))


1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [50]:
df = pd.DataFrame(cvr_text_2000) 
    
# saving the dataframe 
df.to_csv('cvr_text_2000.csv')

In [52]:
cvr_text_3000 = []

for i in range(2000, 3000):
    if i%100 == 0:
        print(i)
    if cvrs.iloc[i,1] == "FALSE":
        cvr_text_3000.append('')
    else:
        cvr_text_3000.append(purposeScrapeSel(cvrs.iloc[i,1]))


2000
2100
2200
2300
2400
2500
2600
2700
2800
2900


In [53]:
df = pd.DataFrame(cvr_text_3000) 
    
# saving the dataframe 
df.to_csv('cvr_text_3000.csv')

In [54]:
cvr_text_4000 = []

for i in range(3000, 4000):
    if i%100 == 0:
        print(i)
    if cvrs.iloc[i,1] == "FALSE":
        cvr_text_4000.append('')
    else:
        cvr_text_4000.append(purposeScrapeSel(cvrs.iloc[i,1]))

3000
3100
3200
3300
3400
3500
3600
3700
3800
3900


In [55]:
df = pd.DataFrame(cvr_text_4000) 
    
# saving the dataframe 
df.to_csv('cvr_text_4000.csv')

In [56]:
cvr_text_5000 = []

for i in range(4000, len(cvrs.index)):
    if i%100 == 0:
        print(i)
    if cvrs.iloc[i,1] == "FALSE":
        cvr_text_5000.append('')
    else:
        cvr_text_5000.append(purposeScrapeSel(cvrs.iloc[i,1]))

4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000


In [57]:
df = pd.DataFrame(cvr_text_5000) 
    
# saving the dataframe 
df.to_csv('cvr_text_5000.csv')

### Combining scraped data into one file

In [58]:
cvr_text = cvr_text_1000 + cvr_text_2000 + cvr_text_3000 + cvr_text_4000 + cvr_text_5000
len(cvr_text)


5079

In [119]:
cvr_text[1:5]

['Sparekassens formål er at drive pengeinstitutvirksomhed samt anden virksomhed, der accessorisk hertil. Sparekassens ønske er desuden at støtte og afholde almennyttige aktiviteter i Sparekassens virkeområde.',
 '',
 '',
 '']

Writing to CSV

In [59]:
df = pd.DataFrame(cvr_text) 
df.to_csv('cvr_text.csv')

Code to re-import cvr_text from csv

In [136]:
import csv

results = []
with open('cvr_text.csv', newline='') as inputfile:
    for row in csv.reader(inputfile):
        results.append(row[1])

results.pop(0)

print(results[1:5])
len(results)

['', 'Sparekassens formål er at drive pengeinstitutvirksomhed samt anden virksomhed, der accessorisk hertil. Sparekassens ønske er desuden at støtte og afholde almennyttige aktiviteter i Sparekassens virkeområde.', '', '']


5080